In [22]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')

In [23]:
# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

run_bg2bw = input("Do you wanna run bg2bw if missing? (y/n)")

# url bedGraph files without a bigwig extra file
bg_url = "/search/?file_format.file_format=bg&type=FileProcessed&extra_files.file_format.display_title%21=bw"

# To Do when run fails it creates the exta file, search for files with bw extra file but status not uploaded or similar
# bg_url = "/search/?file_format.file_format=bg&type=FileProcessed&extra_files.file_format.display_title=bw&extra_files.status="

bg_files = ff_utils.search_metadata(bg_url , key = my_key)

#if beyond run_time, assume failed
run_time = 120

counter = 0
completed = 0
print(len(bg_files))

okay = 0
non = 0
running = 0

chr_size = {"GRCh38": "4DNFI823LSII",
            "GRCm38": "4DNFI3UBJ3HZ",
            "dm6": '4DNFIBEEN92C',
            "galGal5": "4DNFI8MHOUP6"}

for a_bg in bg_files:
    file_id = a_bg['accession']  
    report = get_wfr_out(file_id, 'bedGraphToBigWig',  my_key, md_qc = True, run=run_time)

    if report['status'] == 'running':
        running += 1
        print(file_id, 'still running')
        continue
        
    attributions = get_attribution(a_bg)
    
    if report['status'].startswith('no') or not report:
        non += 1
        if run_bg2bw == 'y':
            print('bg2bw running for', a_bg['accession'])
            # find chr size
            chr_file = chr_size[a_bg['genome_assembly']]
            
            
            inp_f = {'bgfile':file_id, "chromsize": chr_file }
            run_missing_wfr(step_settings('bedGraphToBigWig', 'none', attributions), inp_f, file_id, my_key, my_env, tag="")
        else:
            print('run missing for',file_id)
    else:
        okay += 1
      
print("okay, not okay, running")    
print(okay, non, running)


Do you wanna run bg2bw if missing? (y/n)y
6
bg2bw running for 4DNFI1YUDE7V


bg2bw running for 4DNFI39HH6UD


bg2bw running for 4DNFIG4XW2NB


bg2bw running for 4DNFIEQRK1DK


bg2bw running for 4DNFIJDM948M


bg2bw running for 4DNFI8LGGZM6


okay, not okay, running
0 6 0


In [26]:
# Status update for run and extra file
run_url = '/search/?run_status=complete&type=WorkflowRunAwsem&workflow.title=Converting+bedgraph+to+bigwig&status=in review by lab&limit=all'
runs = ff_utils.search_metadata(run_url , key=my_key)
print(len(runs))

update_run = 0
update_extra = 0
for a_run in runs:
    inp_file = [i['value'] for i in a_run['input_files'] if i['workflow_argument_name'] == "bgfile"][0]
    # update the run status
    if inp_file['status'] != a_run['status']:
        if f_status in ['archived', 'released to project', 'released']:
            print(inp_file['accession'], f_status,"|", a_run['status'])
            ff_utils.patch_metadata({'status': inp_file['status']}, obj_id=a_run['uuid'], key = my_key)
            update_run += 1
        else:
            continue
    # update extra file status
    extra_files = ff_utils.get_metadata(inp_file['accession'], key = my_key, add_on='frame=raw')['extra_files']
    update = False
    for extra_file in extra_files:
        if extra_file['file_format'] == 'd1311111-218e-4f61-aaf0-91f226248b2c':
            if extra_file['status'] != inp_file['status']:
                update = True
                extra_file['status'] = inp_file['status']
    if update:       
            ff_utils.patch_metadata({"extra_files":extra_files}, obj_id=inp_file['accession'], key = my_key)
            update_extra += 1
print(update_extra, 'extra files updated')
print(update_run, 'runs updated')


6
4DNFI8LGGZM6 released to project | in review by lab
4DNFIJDM948M released to project | in review by lab
4DNFIEQRK1DK released to project | in review by lab
4DNFIG4XW2NB released to project | in review by lab
4DNFI39HH6UD released to project | in review by lab
4DNFI1YUDE7V released to project | in review by lab
5 extra files updated
6 runs updated
